In [ ]:
#importing the necessary libray
import pandas as pd
import numpy as np
import os
import cv2

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Lambda, Conv2D, MaxPooling2D, Dropout, Dense, Flatten
from tensorflow.keras.models import load_model

In [ ]:
#Root path for the data and csv file
PATH = r"C:\Users\pradeep reddy\Desktop\data"

In [ ]:
#reading the dataset
df = pd.read_csv(PATH+"\driving_log.csv", names=['center_cam', 'left_cam', 'right_cam', 'steering_angle', 'throttle_value', 'reverse_value', 'speed'])

In [ ]:
#checking the dataset head
df.head()

In [ ]:
#preprocess the image function
def preprocess_the_image(image):
    #reshaping the image
    image = image[60:-25, :, :]
    #resizing the image
    image = cv2.resize(image, (64, 64), cv2.INTER_AREA)
    #converting to RGB color
    image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
    #adding blur to image
    image = cv2.GaussianBlur(image, (3, 3), 0)
    #returing the image
    return image

In [ ]:
def load_image_from_dir(data_dir, image_file):
    #loading the image and then return the image
    return mpimg.imread(os.path.join(data_dir, image_file.strip()))

In [ ]:
#generating batch of data
def batch_gen(PATH, train_X, train_y, batch_size, training):
    #creating empty numpy arrays
    images = np.empty((batch_size, 64, 64, 3))
    steers = np.empty(batch_size) 
    #while loop untill break 
    while 1:
        count = 0
        #loading each batch
        for i in np.random.permutation(train_X.shape[0]):
            #getting three image
            center, left, right = train_X[i]
            #getting sterring angle
            steering_angle = train_y[i]
            
            #loading the images from directory
            image = load_image_from_dir(PATH, center)
            
            # add the image and steering angle to the batch
            images[count] = preprocess_the_image(image)
            steers[count] = steering_angle
            count += 1
            
            #breaking the loop if exceed count
            if count == batch_size:
                break
                
        #yeild the images and steerr angle
        yield images, steers

In [ ]:
#Reading the three images path as input data
X = df[['center_cam', 'left_cam', 'right_cam']].values

#reading the steering angle as the output data
y = df['steering_angle'].values

#splitting the data as train and test set
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
#checking the shape of the train data
print(f'{train_X.shape} {test_X.shape}')

In [ ]:
#checking the shape of the test data
print(f'{train_y.shape} {test_y.shape}')

In [ ]:
# Build model

#sequential model
model = Sequential()

#Adding lambda for the input layer for 
model.add(Lambda(lambda x: x/255.0 - 0.5, input_shape=(64, 64, 3)))

#conv layer for feature extraction
model.add(Conv2D(3, (1, 1), activation='elu'))
model.add(Conv2D(32, (3, 3), activation='elu'))
model.add(Conv2D(32, (3, 3), activation='elu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.35))
model.add(Conv2D(64, (3, 3), activation='elu'))
model.add(Conv2D(64, (3, 3), activation='elu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.35))
model.add(Conv2D(128, (3, 3), activation='elu'))
model.add(Conv2D(128, (3, 3), activation='elu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.35))

#flatting the 3d to 1d 
model.add(Flatten())

#classification process
model.add(Dense(512, activation='elu'))
model.add(Dense(64, activation='elu'))
model.add(Dense(16, activation='elu'))
model.add(Dense(1))

#summary of the model
model.summary()

#compiling the model with mse loss and adam optiizer
model.compile(loss='mse', optimizer=Adam(lr=0.0002))

In [ ]:
#initilizing the model checkpoint 
checkpoint = ModelCheckpoint(
    'my_mod/model-{epoch:03d}.h5',
    monitor='val_loss',
    verbose=0,
    save_best_only=True,
    mode='auto'
)

#training the model
history = model.fit(
    batch_gen(PATH, train_X, train_y, 32, True),
    epochs=100,
    initial_epoch=0,
    steps_per_epoch=200,
    validation_data=batch_gen(PATH, test_X, test_y, 32, False),
    validation_steps=np.ceil(len(test_X) / 32),
    callbacks=[checkpoint],
    verbose=2
)

In [ ]:
model.save("final_model.h5")

In [ ]:
#plotting the loss and val loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['train', 'val'])
plt.title('loss')
plt.xlabel('epoch')